In [ ]:
!pip install --upgrade --quiet  langchain langchain-openai

In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_models import ChatOpenAI

# Load PDF and split into tokens
loaders = [
    PyPDFLoader("//content/drive/MyDrive/Raptor Contract.docx.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Raptor Q&A2.docx.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Robinson Advisory.docx.pdf"),
    PyPDFLoader("/content/drive/MyDrive/Robinson Q&A.docx.pdf")
]
pdf_data = loaders[0].load()  # Access the first loader

text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
split_data = text_splitter.split_documents(pdf_data)

# Set up ChromaDB with vector embeddings
collection_name = "contracts_collection"
local_directory = "contracts_vect_embedding"
persist_directory = os.path.join(os.getcwd(), local_directory)

openai_key = os.environ.get('xxxxxxxxxxxxxxxxxxxxx')
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
vect_db = Chroma.from_documents(
    split_data,
    embeddings,
    collection_name=collection_name,
    persist_directory=persist_directory
)
vect_db.persist()

# Initialize ConversationalRetrievalChain
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chat_qa = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo"),
    vect_db.as_retriever(),
    memory=memory
)

In [20]:
# Example manual evaluation and debugging
user_questions = [
    "What are the terms of payment in the contract?",
    "Can you provide information about the termination clause?",
    "Is there any clause regarding intellectual property rights?"
]

In [21]:
for question in user_questions:
    response = chat_qa({"question": question, "chat_history": []})
    print("User:", question)
    print("RAG System Response:", response["answer"])
    print("Debugging Information:")

User: What are the terms of payment in the contract?
RAG System Response: The terms of payment in the contract specify that the Buyer shall pay, through a Paying Agent, the Closing Cash Consideration to the Sellers, the Warrant Closing Cash Consideration to the Warrantholders, and the Closing Vested Options Cash Consideration to the Trustee for the benefit of the Vested. The payments are to be made in cash by wire transfer of immediately available funds at the Closing.
Debugging Information:
User: Can you provide information about the termination clause?
RAG System Response: I don't have enough context to provide specific information about the termination clause in the contract. The text provided seems to focus on confidentiality and disclosure of information between Sellers and Buyers. If you have a specific section related to termination, please provide that for a more accurate response.
Debugging Information:
User: Is there any clause regarding intellectual property rights?
RAG Syst

In [23]:
user_questions = [
    "What are the terms of payment in the contract?",
    "Can you provide information about the termination clause?",
    "Is there any clause regarding intellectual property rights?"
]

for question in user_questions:
    response = chat_qa({"question": question, "chat_history": []})
    print("User:", question)
    print("RAG System Response:", response["answer"])
    print("Debugging Information:")


User: What are the terms of payment in the contract?
RAG System Response: The terms of payment specified in the contract include the following:
- Closing Cash Consideration to or for the account of Sellers
- Warrant Closing Cash Consideration to or for the account of Warrantholders
- Closing Vested Options Cash Consideration to the Trustee for the benefit of the Vested
Debugging Information:
User: Can you provide information about the termination clause?
RAG System Response: I don't have specific information about the termination clause in the contract based on the context provided.
Debugging Information:
User: Is there any clause regarding intellectual property rights?
RAG System Response: Yes, there are clauses related to intellectual property rights in the contract. These clauses discuss infringement of company IP, disclosure of any interference with company intellectual property, and identification of contractual obligations related to technology and intellectual property.
Debuggin

In [26]:
# Initialize a separate language model for LLM-assisted evaluation
from langchain_community.chat_models import ChatOpenAI

separate_llm = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo")

for question in user_questions:
    # Generate response using the RAG system
    rag_response = chat_qa({"question": question, "chat_history": []})

    # Generate response using the separate language model
    separate_llm_response = separate_llm.invoke(question)  # Using the 'invoke' method to generate response

    # Print the user's question and both responses for comparison
    print("User:", question)
    print("RAG System Response:", rag_response["answer"])
    print("Separate LLM Response:", separate_llm_response)
    # Compare the responses and identify discrepancies


User: What are the terms of payment in the contract?
RAG System Response: The specific payment terms outlined in the contract include the Buyer making payments at the Closing. These payments are to be made in cash by wire transfer of immediately available funds, including the Closing Cash Consideration to Sellers, the Warrant Closing Cash Consideration to Warrantholders, and the Closing Vested Options Cash Consideration to the Trustee for the benefit of the Vested.
Separate LLM Response: content='The terms of payment in the contract are as follows:\n\n1. Payment must be made in full within 30 days of the invoice date.\n2. Late payments will incur a 5% late fee.\n3. All payments must be made in the form of a check or electronic transfer.\n4. Any disputes regarding payments must be resolved within 10 days of notification.\n5. Failure to make payment within the specified timeframe may result in legal action.'
User: Can you provide information about the termination clause?
RAG System Respo

In [ ]:
# Chat loop
chat_history = []

while True:
    query = input('You: ')  # Prompt the user to input a question

    if query.lower() == 'done':  # Check if the user wants to end the conversation
        break

    response = chat_qa({"question": query, "chat_history": chat_history})  # Retrieve response

    chat_history.append({"role": "user", "content": query})  # Update chat history
    chat_history.append({"role": "assistant", "content": response["answer"]})

    print('Assistant:', response["answer"])  # Print the assistant's response